## Programowanie synchroniczne przy użyciu biblioteki _asyncio_

W przeważającej liczbie praktycznych przypadków, programy nie wymagają organizacji w równoległe jednostki obliczeniowe. Często jednak w praktyce wykorzystuje się dużą liczbę żądań IO (_IO_ _requests_), które wymagają czasu na odpowiedź. Biblioteka _asyncio_ ułatwia tworzenie tego typu rozwiązań poprzez tzw. programowanie sterowanie zdarzeniami. W momencie żądania wejścia, wyjścia nie tworzony jest nowy wątek, a jedynie co jakiś czas sprawdzany jest stan żądania. Umożliwia to wątkowi głównemu wykonywanie kolejnych instrukcji. Jeśli żądanie jest zakończone, uruchamiany jest odpowiedni kod obsługujący otrzymane dane. Największą jednak zaletą tego rozwiązania jest brak zmiany kontekstu _context_ _switching_: wskaźnika instrukcji, rejestrów procesora i zmiennych między wątkami, co wpływa na szybkość działania. Poniżej zostaną omówione najważniejsze aspekty programowania asynchronicznego.

## Współprogram (_coroutines_)

Wg. definicji współprogram to ciąg instrukcji, który może zostać zawieszony na czas uruchomienia innego ciągu instrukcji. Najprostszym kodem wykorzystującym tą definicję jest _yield_, który przekazuje sterowanie do funkcji wywołującej, aby następnie powrócić do pierwotnej funkcji. Dodatkowo do samego języka Python wprowadzono dwa nowe słowa kluczowe: _async_ i _await_ umożliwiające zarządzanie współprogramem. Ułatwia to uruchamianie i oczekiwanie na odpowiedź w kodzie asynchronicznym. Funkcja, która wykorzystuje metody asynchronicznych również musi być asynchroniczna. Dlatego już na początkowym etapie życia aplikacji należy wywołać nasz punkt wejścia do programu jako _async_. Służy do tego funkcja _run_ znajdująca się w module _asyncio_.

In [ ]:
from asyncio import sleep, run

async def do_work(work):
    await sleep(work)
    return f'done work in {work}s'

async def main():
    print(await do_work(1))
    print(await do_work(2))
    print('done')

run(main())

W momencie wywołania funkcji wraz z _await_ kod zostanie zawieszony do czasu otrzymania wyniku (linie 5,9,10). W linii 5 została użyta funkcja _sleep_, która zwraca _awaitable_ _object_, obiekt proxy, który umożliwia dalsze przetwarzanie np. oczekiwanie na zakończenie czy anulowanie zadania. Również każda funkcja przed deklaracją której pojawi się słowo kluczowe _async_ zwróci to samo proxy (linia 3). Umożliwia to wywołanie _await_, które obsłuży zdarzenie. Wywołanie funkcji, które nie zawiera oczekiwania na wynik nie zostanie nigdy wywołane. Przykładowo, gdyby w linii 8 pojawiło się samo wywołanie funkcji _do_\__work_ (bez _await_), funkcja nie zostałaby nigdy wywołana.

## Zadania (_task_)

Klasa ta umożliwia asynchroniczne wykonanie kodu i zawiera zestaw przydatnych funkcji jak _cancel_, które anuluje wykonanie zadania (np. gdy trwa zbyt długo czy w momencie gdy użytkownik chce wyjść z aplikacji). Wewnątrz asynchronicznego kodu rzucany jest błąd (_CancelledError_), aby w przypadku oczekiwania na operację _IO_ wznowić działanie kodu na bloku obsługi wyjątku _catch_. Poniżej znajduje się przykład tworzenia dwóch zadań i wykonanie ich asynchronicznie.

In [ ]:
from asyncio import create_task
from time import sleep

async def do_work(work):
    sleep(work) # sleep z biblioteki time również działa
    print(f'done work in {work}s')

async def main():
    task1 = create_task(do_work(1))
    task2 = create_task(do_work(1))
    await task1
    await task2
    print('done')

asyncio.run(main())

## Przydatne funkcje

Programowanie asynchroniczne zyskuje coraz większą popularność. Istnieje wiele modułów, które są kompatybilne z modułem asyncio, a które mogą zastąpić standardowe klasy jak _aiofile_ do obsługi plików, _aiohttp_ do obsługi żądań _htttp_. Z roku na rok przybywa nowych modułów. Jednak w tym miejscu należy podkreślić, że _asyncio_ umożliwia uruchamianie również klasycznych funkcji dostępu do operacji _IO_. Poprzedni przykład kodu (wyżej) wykorzystuje funkcję _sleep_ (linia 5). Poniżej znajduje się lista funkcji z biblioteki standardowej _asyncio_.

### Funkcja _gather_

Umożliwia wywołanie wielu zadań asynchronicznie. Dodatkowo sterowanie zostanie zwrócone dopiero w momencie, gdy każde zadanie zostanie wykonane. Poniżej kod uruchamiający trzy zadania asynchronicznie.

In [ ]:
from asyncio import sleep, run, gather

async def do_work(work):
    await sleep(work)
    print(f'done work in {work}s')
    return f'worker that work {work}s done'

async def main():
    results = await gather(do_work(3), do_work(5), do_work(1))
    [print(result) for result in results]
    print('done')

run(main())

Wyniki zwracane przez _gather_ są sortowane zgodnie z kolejnością występowania w parametrze przekazywanym do funkcji _gather_. 

### Funkcja _wait_

W niektórych przypadkach pożądane jest przetworzenie zakończonej funkcji asynchronicznej zamiast oczekiwać zakończenia wszystkich. Funkcja _wait_ zwraca (bez blokowania) dwuelementową tuplę z listą zadań zakończonych oraz oczekujących. Dodatkowo funkcja posiada parametr _return_\__when_ definiujący kiedy metoda powinna zwrócić wynik. Dostępne opcje to (stałe) _FIRST_\__COMPLETED_, _FIRST_\__EXCEPTION_ i _ALL_\__COMPLETED_. Poniżej znajduje się przykład wykorzystania funkcji _wait_.

In [ ]:
from asyncio import sleep, run, wait, create_task, FIRST_COMPLETED

async def do_work(work):
    await sleep(work)
    print(f'done work in {work}s')
    return f'worker that work {work}s done'

async def main():
    t1 = create_task(do_work(3))
    t2 = create_task(do_work(5))
    t3 = create_task(do_work(1))

    done, _ = await wait([t1, t2, t3], return_when=FIRST_COMPLETED)
    [print(result) for result in done]
    print('one done')
    await wait([t1, t2, t3]) # domyślnie przekazywany jest ALL_COMPLETED
    print('all done')

run(main())

### Przykład użycia _async_ z instrukcją _for_

W przypadku, gdy program często korzysta operacji _IO_ często spotykaną techniką jest zastosowanie instrukcji _yield_, która zwraca wynik do kodu, który wywołał funkcję zwracającą _yield_ i po jego obsłudze sterowanie jest przekazane z powrotem do funkcji, która zwróciła _yield_. Przykładowo poszukujemy pliku, który zawiera pewną frazę. Pliki są przechowywane w usłudze _Amazon_ _s3_. Pobieranie wszystkich plików jest nieefektywne. Dlatego lepiej sprawdzać pliki jeden po drugim i w odpowiednim momencie zakończyć pobieranie. Poniższy przykład prezentuje szablon użycia instrukcji _yield_ wraz z _async_ _for_.

In [ ]:
from asyncio import sleep, run, create_task, wait

async def get_docs():
    for i in range(10):
        yield i #zwracamy zawartość dokumentu
    await sleep(5)

async def main():
    async for doc in get_docs():
        print(doc)
    print('done')

run(main())

Asynchroniczna pętla _for_ wykonywana jest w instrukcji 9. Wynik zwracany jest przed końcem życia współprogramu (linia 5). Słowo _done_ pojawi się dopiero po 5 sekundach, w momencie kiedy zakończy się kod funkcji _get_\__docs_.

## Zadania do wykonania

### 1. Napisz program pobierające dane z Internetu.

Zadanie polega na znalezieniu legalnego źródła mediów w Internecie np. zdjęć. Program powinien posiadać pasek postępu oraz łatwy do zrozumienia system logowania błędów.

In [60]:
import requests
import time
import numpy as np
import progressbar

url = "https://upload.wikimedia.org/wikipedia/commons/2/2f/Naran_random_place.jpg"

def download_file(url, n_chunk=10):
    r = requests.get(url, stream=True)
    block_size = 4096
    file_size = int(r.headers.get('Content-Length', None))
    num_bars = np.ceil(file_size / (n_chunk * block_size))
    bar = progressbar.ProgressBar(maxval=num_bars).start()
    try:
        with open('random-image.jpg', 'wb') as f:
            for i, chunk in enumerate(r.iter_content(chunk_size=n_chunk * block_size)):
                bar.update(i+1)
                time.sleep(0.0005)
                f.write(chunk)
        return 'Download successful'
    except NameError:
        print('Name of value not found')
    except MemoryError:
        print('Size of some variable exceeded')
    except KeyboardInterrupt:
        print('Download was interrupted')
    except ImportError:
        print('Missing some import')
        

download_file(url)

'Download successful'